# Prep data
Use the item-demand-forecast-rossman notebook and data for chapter 1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})

In [ ]:
df = pd.read_csv('../../chapter1/forecasting-api/data/rossman/train.csv')

In [ ]:
def prepare_forecast_data(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.rename(columns= {'Date': 'ds', 'Sales': 'y'})
    df = df[df['Open']==1]
    #df['item_id'] = df['Store']+df['item']
    return df[['ds', 'y', 'Store']]

    
def export_prepared_data(df):
    df.to_csv('train-prepared.csv', date_format='%Y-%m-%d %H:%M:%S', index=False)

In [ ]:
export_prepared_data(prepare_forecast_data(df))

# Connections

In [ ]:
import boto3

In [ ]:
# Connect API session
session = boto3.Session(region_name='eu-west-1') 
forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

In [ ]:
# check you can communicate with Forecast API session
forecast.list_predictors()

# Train forecaster
Assume that data has been uploaded and setup complete (see the bash scripts ...)

Leverage

1. [Amazon Forecast Samples | Building Your Predictor](https://github.com/aws-samples/amazon-forecast-samples/blob/master/notebooks/basic/Tutorial/2.Building_Your_Predictor.ipynb)


In [ ]:
algorithm_arn = 'arn:aws:forecast:::algorithm/Prophet'

predictor_name = f"store_demand_prophet"
print(f"Predictor Name = {predictor_name}")

In [ ]:
dataset_group_arn = "arn:aws:forecast:eu-west-1:508972911348:dataset-group/store_demand_group" # to be stored as secret
DATASET_FREQUENCY = 'D'
create_predictor_response = \
    forecast.create_predictor(PredictorName=predictor_name,
                              AlgorithmArn=algorithm_arn,
                              ForecastHorizon=7,
                              PerformAutoML=False,
                              PerformHPO=False,
                              InputDataConfig= {"DatasetGroupArn": dataset_group_arn},
                              FeaturizationConfig= {"ForecastFrequency": DATASET_FREQUENCY}
                             )





In [ ]:
predictor_arn = create_predictor_response['PredictorArn']

In [ ]:
predictor_arn

## Create a Forecast

In [ ]:
forecast_name = f"store_demand_forecast"
print(f"Forecast Name = {forecast_name}")



In [ ]:


# create_forecast_response=forecast.create_forecast(ForecastName=forecastName,
#                                                   PredictorArn=predictor_arn)
# forecast_arn = create_forecast_response['ForecastArn']

create_forecast_response = \
    forecast.create_forecast(ForecastName=forecast_name,
                             PredictorArn=predictor_arn)

forecast_arn = create_forecast_response['ForecastArn']



## Query Forecast 

This only works once the forecaster is 'ACTIVE'.

In [ ]:
# check forecast status
forecast.describe_forecast(ForecastArn=forecast_arn)

In [ ]:
forecast_status = forecast.describe_forecast(ForecastArn=forecast_arn)


In [ ]:
import datetime
forecast_status['LastModificationTime']>datetime.datetime.now()

In [ ]:
forecast_response = forecastquery.query_forecast(
    ForecastArn=forecast_arn,
    Filters={"item_id": "1"})

forecast_response